In [4]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import islice
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, cohen_kappa_score, matthews_corrcoef
from keras.utils import to_categorical
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import itertools

ModuleNotFoundError: No module named 'sklearn.ensemble'

# Adatbeolvasás

## Patientdata
Az eredeti csv feldolgozása már tanulásra előkészített adatokkal

In [ ]:
readIn = pd.read_csv("data/df_patientdata.csv")
df_patientdata= pd.DataFrame(readIn)

In [ ]:
df_patientdata.head(3)

## Image data
A dicom fájlokból egységesített, normalizált képek. 
- Numpy tömbben
- Méretük: (256,256)
- Minden pixelhez csak egy érték tartozik, értékük 0-1 között van

Disclaimer!! Ez pár percig el fog tartani

# Effective data loading

Load and train models in batches. The prepared data is stored in `data/images_splitX.npz`.

There are 12 data files:
- **0-8** for training the **individual models**
- **9-10** for training the **Voting classifier**
- **11** for **testing and evaluating**

The saved data is already shuffled.

In [ ]:
split_num=9

In [ ]:
# Read 
def getXandY(splitNum):
    base_name='data/images'
    split_name = f'{base_name}_split{splitNum}.npz'
    data = np.load(split_name, allow_pickle=True)
    data_images=data['images']
    data_ids=data['ids']
    print(f'Merging         {splitNum}', end='\r')
    image_data=[]
    for id, image in zip(data_ids, data_images):
        image_data.append([id, image.flatten()])

    del data_images
    del data_ids

    df_images =pd.DataFrame(image_data, columns=['PatientID', 'Image'])

    del image_data
    
    merged_df = pd.merge(df_patientdata, df_images, on='PatientID', how='inner')

    X = merged_df.drop(['PatientID', 'Pathology'], axis=1)
    y = merged_df['Pathology']
    return X, y

# Training

## Individual model training

In [ ]:
class IndividualModelTrainer:
    def __init__(self, split_num):
        self.split_num = split_num
        self.randomForest_i = RandomForestClassifier(n_estimators=20, warm_start=True)
        self.randomForest_pd = RandomForestClassifier(n_estimators=20, warm_start=True)
        self.gradBoosting_pd = GradientBoostingClassifier(n_estimators=30, warm_start=True)
        self.nn_i = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=1000, random_state=42)
        self.nn_initialized = False
        
    def training_for_sklearn(self):
        base_name='data/images'
        for i in range(split_num):
            print(f'Processing      {i}', end='\r')
            X_train, y_train = getXandY(i)
        
            X_image_train = X_train['Image'].tolist()
            X_patient_data_train = X_train.drop(['Image'], axis=1)
        
            print(f'Training images {i}', end='\r')
            self.train_image_classifiers(X_image_train, y_train)
            print(f'Training csv    {i}', end='\r')
            self.train_patient_data_classifiers(X_patient_data_train, y_train)

    def train_image_classifiers(self, X_image_train, y_train):
        self.randomForest_i.fit(X_image_train, y_train)
        self.randomForest_i.n_estimators+=20
        if self.nn_initialized:
            self.nn_i.partial_fit(X_image_train, y_train)
        else:
            self.nn_i.partial_fit(X_image_train, y_train, classes=[0, 1, 2, 3])
            self.nn_initialized = True
        

    def train_patient_data_classifiers(self, X_patient_data_train, y_train):
        self.randomForest_pd.fit(X_patient_data_train, y_train)
        self.gradBoosting_pd.fit(X_patient_data_train, y_train)
        self.randomForest_pd.n_estimators+=3
        self.gradBoosting_pd.n_estimators+=6

In [ ]:
individualModelTrainer = IndividualModelTrainer(split_num)

In [ ]:
individualModelTrainer.training_for_sklearn()

## Voting classifier

### Load ensemble data

In [ ]:
X_train_ensemble, y_train_ensemble = getXandY(9)

### Create voting classifier

In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, imageSelector= False, isNn=False):
        self.imageSelector = imageSelector
        self.isNn = isNn

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.imageSelector:
            return X['Image'].tolist()
            #if self.isNn:
                #return np.asarray(images)
            #return images
        return X.drop(['Image'], axis=1)

In [ ]:
patient_data_transformer = ColumnSelector()
image_transformer = ColumnSelector(imageSelector=True)

In [ ]:
estimators = [
    ('gb_patient_data', Pipeline([('selector', patient_data_transformer), ('model', individualModelTrainer.gradBoosting_pd)])),
    ('rf_patient_data', Pipeline([('selector', patient_data_transformer), ('model', individualModelTrainer.randomForest_pd)])),
    ('rf_image', Pipeline([('selector', image_transformer), ('model', individualModelTrainer.randomForest_i)])),
    ('nn_image', Pipeline([('selector', image_transformer), ('model', individualModelTrainer.nn_i)])),
    # Add more models if desired
]

In [ ]:
voting_classifier = VotingClassifier(estimators, voting='soft')

### Train voting classifier

In [ ]:
voting_classifier.fit(X_train_ensemble, y_train_ensemble)

# Load test data

In [ ]:
X_test, y_test = getXandY(11)

# Evaluation

Teljesítményértékelő metrikák többosztályos klasszifikációs problémákhoz:

    - Confusion Matrix (--> accuracy, precision, recall, F1 score)
    - ROC-AUC
    
ToDo a végleges leadásra: vizualizáció

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
def plot_roc_curve(y_true, y_pred_probs, classes, xlim=None, ylim=None):
    """
    This function plots the ROC curve for each class and allows zooming in on a specific area.
    """
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(len(classes)):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(10, 6))
    lw = 2
    colors = ['darkorange', 'green', 'blue', 'red', 'purple']  # Adjust the colors based on the number of classes

    for i, color in zip(range(len(classes)), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC ({}), area = {:.2f}'.format(classes[i], roc_auc[i]))

    # Set custom axis limits if provided
    if xlim:
        plt.xlim(xlim)
    else:
        plt.xlim([0.0, 1.0])

    if ylim:
        plt.ylim(ylim)
    else:
        plt.ylim([0.0, 1.05])

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    print(np.unique(y_pred))
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(cm, classes=["Class 0", "Class 1", "Class 2", "Class 3"], normalize=True)

    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Precision, Recall, F1 Score
    precision = precision_score(y_test, y_pred, average='weighted')
    
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # ROC-AUC
    y_pred_probs = model.predict_proba(X_test)
    roc_auc = roc_auc_score(y_test, y_pred_probs, multi_class='ovr', average='weighted')
    
    y_true_binary = label_binarize(y_test, classes=[0, 1, 2, 3]) 
    plot_roc_curve(y_true_binary, y_pred_probs, classes=["Class 0", "Class 1", "Class 2", "Class 3"])

    # Example usage with custom axis limits
    # Adjust the values of xlim and ylim as needed
    # For example, xlim=(0.0, 0.5), ylim=(0.6, 1.0)
    # If not specified, the default limits will be used
    plot_roc_curve(y_true_binary, y_pred_probs, classes=["Class 0", "Class 1", "Class 2", "Class 3"], xlim=(0.0, 0.5), ylim=(0.6, 1.0))

    # Cohen's Kappa
    kappa = cohen_kappa_score(y_test, y_pred)

    # Matthew's Correlation Coefficient
    mcc = matthews_corrcoef(y_test, y_pred)
    
    print("Accuracy: {:.4f}".format(accuracy))
    print("Precision: {:.4f}".format(precision))
    print("Recall: {:.4f}".format(recall))
    print("F1 Score: {:.4f}".format(f1))
    print("ROC-AUC: {:.4f}\n".format(roc_auc))
    print("Classification Report:\n", classification_report(y_test, y_pred, target_names=["Class 0", "Class 1", "Class 2", "Class 3"]))
    print("Cohen's Kappa: {:.4f}".format(kappa))
    print("Matthew's Correlation Coefficient: {:.4f}".format(mcc))

In [ ]:
evaluate_model(voting_classifier, X_test, y_test)

# Modell kimentése a Frontendhez

In [ ]:
joblib.dump(voting_classifier, 'voting_classifier.joblib')